In [1]:


# Using spacy.load().
import spacy
nlp = spacy.load("en_core_web_trf")

# Importing as module.
import en_core_web_trf
nlp = en_core_web_trf.load()

In [2]:
import spacy
nlp = spacy.load("en_core_web_trf")

In [3]:
pip install xlsxwriter

In [4]:
pip install openpyxl

In [5]:
import nltk
import random
import spacy
import sys
sys.path.append('../input')
import itertools
from itertools import groupby
from operator import itemgetter
from xuexibao.utils.fileUtils import *
random.seed('data')
nlp = spacy.load('en_core_web_trf')

from nltk.corpus import stopwords
from sklearn.feature_extraction._stop_words import ENGLISH_STOP_WORDS

CUSTOM_STOP_WORDS = set(['app','allow','facebook','youtube','twitter','spotify','instagram','tiktok'])
NOT_STOP_WORDS = set(['your', 'own','take'])
MY_STOP_WORDS = set(stopwords.words('english')).union(set(ENGLISH_STOP_WORDS)).union(CUSTOM_STOP_WORDS) - NOT_STOP_WORDS


In [6]:
def syntactic_match(sentence):
    
    sentence = sentence.replace('&amp;','and')
    doc = nlp(sentence)

    features = {}

    features['NP'] = features.get('NP',[])
    features['CC_NOUN'] = features.get('CC_NOUN',[])
    features['CC_ADJ'] = features.get('CC_ADJ',[])
    features['DOBJP'] = features.get('DOBJP',[])
    features['DOBJ'] = features.get('DOBJ',[])
    features['POBJ'] = features.get('POBJ',[])

    for np in doc.noun_chunks:
        if len(np) > 1:
            # print('NP:',np,[item.pos_ for item in np],[np.start,np.end-1])

            features['NP'].append([tuple(item.lemma_ for item in np),
                                       tuple(item.pos_ for item in np),
                                       tuple(item.i for item in np)])

    for token in doc:
        # print(token.i, token.text, token.pos_, token.dep_, token.head.text, token.head.pos_, [child for child in token.children])
        tmp_list = []

        if token.pos_ == 'CCONJ' and token.head.pos_ == 'NOUN':
            tmp_list.append(token.head)
            tmp_list += list(token.head.conjuncts)
            ccnoun_list = []
            for word in tmp_list:
                ccnoun_list.extend(list(filter(lambda item:item.i <= word.i, [child for child in word.children])) + [word])
            ccnoun_list.append(token)
            ccnoun_list = sorted(ccnoun_list,key = lambda item:item.i)
            # print('CC_NOUN:', ccnoun_list, [item.pos_ for item in ccnoun_list], [ccnoun_list[0].i, ccnoun_list[-1].i])

            features['CC_NOUN'].append([tuple(item.lemma_ for item in ccnoun_list),
                                       tuple(item.pos_ for item in ccnoun_list),
                                       tuple(item.i for item in ccnoun_list)])

        if token.pos_ == 'CCONJ' and token.head.pos_ == 'ADJ':
            tmp_list.append(token.head)
            tmp_list += list(token.head.conjuncts)
            ccadj_list = []
            for word in tmp_list:
                ccadj_list.extend([word])
            ccadj_list.append(token)
            ccadj_list = sorted(ccadj_list,key = lambda item:item.i)
            # print('CC_ADJ:', ccadj_list, [item.pos_ for item in ccadj_list], [ccadj_list[0].i, ccadj_list[-1].i])

            features['CC_ADJ'].append([tuple(item.lemma_ for item in ccadj_list),
                                       tuple(item.pos_ for item in ccadj_list),
                                       tuple(item.i for item in ccadj_list)])


        if token.dep_ == 'pobj':
            pnouns = list(token.conjuncts) + [token]
            for pnoun in pnouns:
                tmp_list = list(filter(lambda item:item.i <= pnoun.i, token.ancestors)) + [pnoun] + list(filter(lambda item:item.i <= pnoun.i, pnoun.children))
                tmp_list = sorted(tmp_list,key = lambda item:item.i)

                for idx,word in enumerate(tmp_list[::-1]):
                    if word.pos_ == 'VERB':
                        
                        tmp_list = tmp_list[-(idx+1):]

                        # print('POBJ:',tmp_list, [tmp_list[0].i,tmp_list[-1].i])

                        features['POBJ'].append([tuple(item.lemma_ for item in tmp_list),
                                           tuple(item.pos_ for item in tmp_list),
                                           tuple(item.i for item in tmp_list)])
                        break
                        
        if token.dep_ == 'nsubjpass':
            tmp_list += [child for child in token.children]
            tmp_list.append(token)
            tmp_list = list(filter(lambda item:item.i <= token.i, tmp_list))
            tmp_list = sorted(tmp_list,key = lambda item:item.i)
            tmp_list = [token.head] + tmp_list
            if len(tmp_list)>1:
                # print('DOBJP:',tmp_list,[item.lemma_ for item in tmp_list],[item.pos_ for item in tmp_list],[tmp_list[0].i,tmp_list[-1].i])

                features['DOBJP'].append([tuple(item.lemma_ for item in tmp_list),
                                           tuple(item.pos_ for item in tmp_list),
                                           tuple(item.i for item in tmp_list)])


        if token.dep_ == 'dobj':
            tmp_list += [child for child in token.children]
            tmp_list.append(token)
            tmp_list = list(filter(lambda item:item.i <= token.i, tmp_list))
            tmp_list.append(token.head)
            tmp_list = sorted(tmp_list,key = lambda item:item.i)
            if len(tmp_list)>1:
                # print('DOBJ:',tmp_list,[item.lemma_ for item in tmp_list],[item.pos_ for item in tmp_list],[tmp_list[0].i,tmp_list[-1].i])
                pass

            if  tmp_list[0].pos_ == 'VERB' and tmp_list[-1].pos_ in ['NOUN','PROPN','PRON']:
                verb_list = [item for item in tmp_list[0].conjuncts if item.pos_ == 'VERB'] + [tmp_list[0]]
                noun_list = [tmp_list[-1]] + list(tmp_list[-1].conjuncts)
                noun_list = [list(filter(lambda item:item.i <= token.i, [child for child in token.children])) + [token] for token in noun_list]
                # print(verb_list, noun_list)
                result_list = []
                for verb in verb_list:
                    for noun in noun_list:
                        result_list.append([verb] + noun)
                for item in result_list:
                    # print(item, [token.pos_ for token in item], [item[0].i, item[-1].i])
                    features['DOBJ'].append([tuple(token.lemma_ for token in item),
                                       tuple(token.pos_ for token in item),
                                       tuple(token.i for token in item)])
    
    return features

In [7]:
def split_conj(feature_list):
    words, pos, idxs = feature_list
    result_list = []
    for k, g in groupby(enumerate(idxs), lambda ix : ix[0] - ix[1]):
        idx = list(map(itemgetter(0), g))
        start = idx[0]
        end = idx[-1] + 1
        result_list.append([words[start:end], pos[start:end], idxs[start:end]])
    return result_list

def pdobj(features):
    pobj = features['POBJ']
    dobj = features['DOBJ']
    for pitem in pobj:
        pitem_idx = pitem[2]
        for ditem in dobj:
            ditem_idx = ditem[2]
            if pitem_idx[0] == ditem_idx[0] and (set(pitem_idx).issubset(ditem_idx) or set(ditem_idx).issubset(pitem_idx)) == False:
                # print(pitem_idx, ditem_idx)
                new_ditem = [list(a)+list(b[1:]) for a,b in zip(pitem, ditem)]
                x_list,y_list,z_list = [],[],[]
                for z,x,y in sorted(zip(new_ditem[2], new_ditem[0], new_ditem[1])):
                    if z not in z_list:
                        x_list.append(x)
                        y_list.append(y)
                        z_list.append(z)
                new_ditem_sort = [tuple(x_list), tuple(y_list), tuple(z_list)]
                features['PDOBJ'] = features.get('PDOBJ',[])
                features['PDOBJ'].append(new_ditem_sort)
    return features

def delete_start_stop(feature_item):
    w,p,i = feature_item
    w = tuple(map(lambda x: x.lower(),w))
    if not (set(w) - MY_STOP_WORDS):
        return None
    for idx,word in enumerate(w):
        if word not in MY_STOP_WORDS:
            new_item = [w[idx:],p[idx:],i[idx:]]
            break
    return new_item

def delete_features(features):
    candidate_list = []
    delete_list = []
    for item in features.values():
        for candidate in item:
            clean_candidate = delete_start_stop(candidate)
            if clean_candidate:
                candidate_list.append(clean_candidate)
    for item in itertools.combinations(candidate_list, 2):
        if set(item[0][2]).issubset(item[1][2]):
            delete_list.append(item[0])
        elif set(item[1][2]).issubset(item[0][2]):
            delete_list.append(item[1])

    new_features = []
    for value in candidate_list:
        if (len(value[1]) == 1) and (value[1] != ('VERB')):
            continue
        if value not in delete_list:
            new_features.append(value)
    return new_features

In [9]:
def fiter(lines):
    new_lines = []
    for i in lines:
        if len(i[0])>60 and len(i[0])<600 and "Incredible" not in i[0] and i not in new_lines:
            new_lines.append(i)
    return new_lines
def patern(test_lines,name):
    json_list = []
    for ridx,line in enumerate(test_lines):
        sents = nltk.sent_tokenize(line[0])
        for sidx,sent in enumerate(sents):
            result_dict = {'review_id':ridx,
                            'sent_id':sidx,
                            'sent':sent,}
            features = syntactic_match(sent)

            new_cc_list = []
            for item in features['CC_NOUN']:
                new_cc_list.extend(split_conj(item))
            features['CC_NOUN'] = new_cc_list

            features = pdobj(features)
            new_features = delete_features(features)
            result_dict['features'] = new_features
            json_list.append(result_dict)
    dumpJson('./{}.json'.format(name),json_list)

In [11]:
import random
random.seed('data')
lines = readCsvToList('../input/privacy-datas/Privacy_data/Instagram_privacy.csv')
app_list = [
"Facebook_privacy",
"Instagram_privacy",
"Spotify_privacy",
"TikTok_privacy",
"Twitter_privacy",
"YouTube_privacy",
"pocketguard"
]

for i in app_list:
    lines = readCsvToList('../input/privacy-datas/Privacy_data/{}.csv'.format(i))
    a = fiter(lines)[:500]
    random.shuffle(a)
    test_lines = a[:50]
    patern(test_lines,i+'_syntactic')